In [1]:
using Plots

In [1]:
include("everest_option.jl")

everest_option_monte_carlo (generic function with 2 methods)

In [2]:
function compare_variance(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
                        S₀::Array{Float64},  sigma::Array{Float64}, correlation_matrix::Array{Float64})


    normal = everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix,"basic")
    antithetic = everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix,"antithetic")
    quasi = everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix,"quasi_monte_carlo")
    moment_matching = price_everest_moment_matching(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix)
    LHS = everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix,"LHS")

        
    mean_elements = normal[1]
    bottom_elements = normal[2]
    upper_elements = normal[3]

    mean_antithetic = antithetic[1]
    bottom_antithetic = antithetic[2]
    upper_antithetic = antithetic[3]

    mean_quasi = quasi

    mean_moment_matching = moment_matching[1]
    bottom_moment_matching = moment_matching[2]
    upper_moment_matching = moment_matching[3]

    mean_LHS = LHS[1]
    bottom_LHS = LHS[2]
    upper_LHS = LHS[3]

    length_normal = upper_elements-bottom_elements
    length_antithetic = upper_antithetic - bottom_antithetic
    length_moment_matching = upper_moment_matching - bottom_moment_matching
    length_LHS = upper_LHS-bottom_LHS

    return [mean_elements, mean_antithetic, mean_moment_matching, mean_LHS, mean_quasi, 
            length_normal, length_antithetic, length_moment_matching, length_LHS]
end


function num_of_sims(num_of_sims::Array{Int}, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
    S₀::Array{Float64},  sigma::Array{Float64}, correlation_matrix::Array{Float64})

    results::Matrix{Float64} = zeros(length(num_of_sims), 9)
    
    for (ind, num_of_sim) in enumerate(num_of_sims)
        results[ind,:] = compare_variance(num_of_sim, α, T, N, r, C, basket_volume, S₀,   sigma, correlation_matrix)
    end

    return results

end 

function mu_value(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
                S₀::Array{Float64},  sigma::Array{Float64}, correlation_matrix::Array{Float64})


    results::Matrix{Float64} = zeros(length(mu), 9)
    for (ind, m) in enumerate(mu)
        results[ind,:] = compare_variance(num_of_sim, α,T,N, r, C, basket_volume, S₀, [m for i in 1:basket_volume], sigma, correlation_matrix)
    end

    return results

end

function interest_rate(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Array{Float64}, C::Float64, basket_volume::Int, 
                    S₀::Array{Float64},  sigma::Array{Float64}, correlation_matrix::Array{Float64})


    results::Matrix{Float64} = zeros(length(rs), 9)
    for (ind, r) in enumerate(rs)
    results[ind,:] = compare_variance(num_of_sim, α,T,N, r, C, basket_volume, S₀,   sigma, correlation_matrix)
    end

    return results

end 


function maturity(num_of_sim::Int, α::Float64,T::Int, r::Float64, C::Float64, basket_volume::Int, 
                    S₀::Array{Float64},  sigma::Array{Float64}, correlation_matrix::Array{Float64})

    results::Matrix{Float64} = zeros(T, 9)
    for maturity in 1:T
        results[maturity,:] = compare_variance(num_of_sim, α, maturity, maturity*250, r, C, basket_volume, S₀,   sigma, correlation_matrix)
    end

    return results

end


function correlation(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
    S₀::Array{Float64},  sigma::Array{Float64})

    corr_matrices::Vector{Matrix{Float64}} = [[1.0 0.1 0.1; 0.1 1.0 0.1; 0.1 0.1 1.0], [1.0 0.5 0.4; 0.5 1.0 0.6; 0.4 0.6 1.0], [1.0 0.9 0.8; 0.9 1.0 0.7; 0.8 0.7 1.0]] 


    results::Matrix{Float64} = zeros(3, 9)
    for correlation in eachindex(corr_matrices)
        results[correlation,:] = compare_variance(num_of_sim, α,T,N, r, C, basket_volume, S₀,   sigma, corr_matrices[correlation])
    end
    return results

end


function plot_variance(vals, variance, title, xlabel)
    scatter(vals,variance, label = ["normal" "antithetic" "moment_matching" "LHS"], title = title, legend = :outertop)
    xlabel!(xlabel)
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\", title, "_everest_", ".svg"))
end

function plot_mean(vals, mean, title, xlabel)
    scatter(vals, mean, label=["mean_elements" "mean_antithetic" "mean_moment_matching" "mean_LHS" "mean_quasi"], legend = :outertop)
    title!(title)
    xlabel!(xlabel)
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\", title,"_everest_", ".svg"))
end

function plot_bar_plot(results,name)
    bar(["słaba", "średnia", "silna"],results,label=name,title=string("długość przedziału ~ korelacja dla metody ",name), xlabel = "siła korelacji między spółkami", xticks = :all,legend=false, fmt = :svg)    
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\", name, "_everest_", ".svg"))
end

plot_bar_plot (generic function with 1 method)

In [3]:
cov_matrix = [1.0 0.3 0.4; 0.3 1.0 0.1; 0.4 0.1 1.0] 

3×3 Matrix{Float64}:
 1.0  0.3  0.4
 0.3  1.0  0.1
 0.4  0.1  1.0

In [4]:
results = compare_variance(1000,0.01,2,500,0.03,8.0,3,[10.0,10.0,10.0],[0.15,0.15,0.15],cov_matrix)[1:5]

5-element Vector{Float64}:
 16.037312058246485
 15.99549474854914
 17.534116268673994
 17.311628640607335
 17.435958684317846

In [5]:
(results[2:5] .- results[1])./results[1]

4-element Vector{Float64}:
 -0.002607501153900811
  0.09333261116271933
  0.07945948658557084
  0.08721203534554711

# Czas wykonania

In [5]:
function compare_pricing_time(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
                                S₀::Array{Float64},  sigma::Array{Float64}, correlation_matrix::Array{Float64})


    t_normal = @elapsed [everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix, "basic") for i in 1:1000]
    
    t_antithetic = @elapsed  [everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix,"antithetic") for i in 1:1000]
    
    t_quasi_monte_carlo = @elapsed [everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix, 
                                                                "quasi_monte_carlo") for i in 1:1000]
    
    t_moment_matching = @elapsed [price_everest_moment_matching(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix,) for i in 1:1000] 
    
    t_LHS = @elapsed [everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, sigma, correlation_matrix, "LHS") for i in 1:1000]

    return Dict("basic" => t_normal, "antithetic" => t_antithetic, "quasi_monte_carlo" => t_quasi_monte_carlo,
                "moment_matching" => t_moment_matching, "LHS" => t_LHS)
end


compare_pricing_time (generic function with 1 method)

In [6]:
time_of_execution = compare_pricing_time(1000,0.01,2,500,0.03,8.0,3,[10.0,10.0,10.0],[0.15,0.15,0.15],cov_matrix)

Dict{String, Float64} with 5 entries:
  "moment_matching"   => 240.709
  "LHS"               => 301.483
  "basic"             => 229.516
  "quasi_monte_carlo" => 4.75098
  "antithetic"        => 537.216

# Liczba symulacji a szerokość przedziału

In [5]:
n_of_sims = [100:10:2000;]
sims::Matrix{Float64} = num_of_sims(n_of_sims,0.01,2,500,0.03,8.0,3,[10.0,10.0,10.0],[0.15,0.15,0.15],cov_matrix)
variance = sims[:,6:end]
plot_variance(n_of_sims,variance,"długość przedziału ufności ~ wielkość próby", "wielkość próby")
means = sims[:,1:5]
plot_mean(n_of_sims,means,"cena opcji ~ wielkość próby", "wielkość próby")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\cena opcji ~ wielkość próby_everest_.svg"

# Stopy procentowe

In [6]:
rs = [0.01:0.001:0.1;]
sims_rs::Matrix{Float64} = interest_rate(1000,0.01,2,500,rs,8.0,3,[10.0,10.0,10.0],[0.15,0.15,0.15],cov_matrix)
variance_rs = sims_rs[:,6:end]
plot_variance(rs,variance_rs, "długość przedziału ufności ~ stopy procentowe", "stopy procentowe")
means_rs = sims_rs[:,1:5]
plot_mean(rs,means_rs, "cena opcji ~ stopy procentowe", "stopy procentowe")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\cena opcji ~ stopy procentowe_everest_.svg"

# Matura

In [7]:
sims_maturity::Matrix{Float64} = maturity(1000,0.01,10,0.03,8.0,3,[10.0,10.0,10.0],[0.15,0.15,0.15],cov_matrix)
mat= [i for i in 1:10]
variance_maturity = sims_maturity[:,6:end]
plot_variance(mat,variance_maturity, "długość przedziału ~ matura", "matura")
means_maturity = sims_maturity[:,1:5]
plot_mean(mat,means_maturity, "cena opcji ~ matura", "matura")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\cena opcji ~ matura_everest_.svg"

# Korelacja

In [18]:
sims_correlation::Matrix{Float64} = correlation(1000,0.01,2,500,0.03,8.0,3,[10.0,10.0,10.0],[0.15,0.15,0.15])
plot_bar_plot(sims_correlation[:,6],"basic")
plot_bar_plot(sims_correlation[:,7],"antithetic")
plot_bar_plot(sims_correlation[:,8],"moment_matching")
plot_bar_plot(sims_correlation[:,9],"LHS")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\LHS_everest_.svg"